In [52]:
import pandas as pd
import requests
import numpy as np
from itertools import product
from pathlib import Path
from glob import glob

In [2]:
PATH_DATA = Path("data")

In [55]:
class DataAcquisition:
    
    def __init__(self, path_data, country_site='MCO'):
        """
        NOTA: Máximo offset permitido por la API de MercadoLibre es 1000

        Args:
            path_data (Path): Ubicación donde se guardará la data extraída.
            country_site (str, optional): Identificador del país para extraer data. Defaults to 'MCO'.
        """
        self.path_data = path_data
        self.country_site = country_site
        self.cats = self.get_categories()
        
    def get_categories(self):
        cats = requests.get(f'https://api.mercadolibre.com/sites/{self.country_site}/categories')
        cats = pd.DataFrame(cats.json())
        cats = cats.sort_values(by = "id").reset_index(drop = True)
        return cats
    
    @staticmethod
    def get_only_one_attr(row, attribute):
        for i in range(len(row)):
            if row[i]["id"] == attribute:
                return row[i]
        return None    
    
    def items_by_category(self, category, offset):
        items = requests.get(f'https://api.mercadolibre.com/sites/{self.country_site}/search?category={category}&offset={offset}')
        items = pd.DataFrame(items.json()['results'])
        items['category'] = category
        items = items.dropna(axis=1, how='all')
        
        if items.shape[0] == 0:
            return items
                
        # Keep only brand attributes
        if 'attributes' in items.columns:
            items["brand"] = items["attributes"].apply(lambda x: self.get_only_one_attr(x, "BRAND"))
        # Generate new cols for item condition and brand
        extra_cols = self.explode_data(items)
        items = pd.concat([items, extra_cols], axis = 1)
        items = self.clean_data_cols(items)
        
        return items
    
    def get_all_items_cats(self, init_offset, final_offset):
        
        offset_range = np.arange(init_offset, final_offset+50, 50)
        for category, offset in product(self.cats['id'], offset_range):
            cat_data = self.items_by_category(category, offset)
            if cat_data.shape[0] == 0:
                continue
            # Saving extracted data
            self.save_items(self.path_data, category, offset, cat_data)
    
    def explode_data(self, df_items):
        cols_explode = ['shipping', 'seller', 'installments', 'brand']
        df_explode_final = pd.DataFrame()
        for col in cols_explode:
            if col not in df_items.columns:
                continue
            df_explode = df_items[col].apply(pd.Series)
            df_explode.columns = [col + "_" + str(new_col) for new_col in df_explode.columns]
            df_explode_final = pd.concat([df_explode_final, df_explode], axis = 1)
        return df_explode_final
    
    @staticmethod
    def save_items(path, category, offset, items):
        items.to_csv(path / f'data_items_{category}_{offset}.csv', index=False)
        
    @staticmethod
    def clean_data_cols(df):
        cols_delete = ['thumbnail_id', 'thumbnail', 'currency_id', 'order_backend',
                       'use_thumbnail_id', 'attributes', 'installments', 
                       'differential_pricing', 'inventory_id', 'variation_filters',
                       'variations_data', "shipping", "seller", "brand_id",
                       "brand_name", "brand_value_id", "brand_attribute_group_id",
                       "brand_attribute_group_name", "brand_value_struct", "brand_values",
                       "brand_source", "brand_value_type", "brand"]
        existing_columns = [col for col in df.columns if col in cols_delete]
        return df.drop(columns = existing_columns)
    
    @staticmethod
    def compile_data(path):
        files = list(path.glob('*.csv'))
        data = pd.concat([pd.read_csv(file) for file in files], ignore_index=True)
        return data
        

In [49]:
data = DataAcquisition(path_data = PATH_DATA)
# En caso de requerir continuar con la ejecución del proceso
# desde cierta categoría en específico, se puede hacer uso
# del siguiente código para eliminar las categorías ya procesadas
# last_cat_index = prueba.cats[prueba.cats["id"]=="MCO40433"].index[0]
# prueba.cats = prueba.cats[prueba.cats.index >= last_cat_index]
data.get_all_items_cats(0, 1000)

In [56]:
data = DataAcquisition(path_data = PATH_DATA)

compiled_data = data.compile_data(PATH_DATA)

In [57]:
compiled_data

,id,title,condition,catalog_product_id,listing_type_id,permalink,buying_mode,site_id,category_id,domain_id,...,shipping_promise,seller_id,seller_nickname,installments_quantity,installments_amount,installments_rate,installments_currency_id,brand_value_name,location,seller_contact
0,MCO1324022622,Pila Recargables Aa X2 Energizer 2000 Mah,new,MCO21850181,gold_special,https://www.mercadolibre.com.co/pila-recargabl...,buy_it_now,MCO,MCO7279,MCO-CELL_BATTERIES,...,NaN,199769978,OPERADOR TO,36.0,647.47,0.0,COP,Energizer,NaN,NaN
1,MCO1271613775,Pila Energizer Recharge Universal Aaa X 4und,new,MCO22015422,gold_special,https://www.mercadolibre.com.co/pila-energizer...,buy_it_now,MCO,MCO7279,MCO-CELL_BATTERIES,...,NaN,199769978,OPERADOR TO,36.0,1039.64,0.0,COP,Energizer,NaN,NaN
2,MCO1349719667,Audífonos In-ear Inalámbricos Bluetooth F9-5 N...,new,MCO16224063,gold_special,https://www.mercadolibre.com.co/audifonos-in-e...,buy_it_now,MCO,MCO3697,MCO-HEADPHONES,...,NaN,1449066564,ROJASMARIA20230811205706,36.0,554.17,0.0,COP,Shenzhen Yihaotong,NaN,NaN
3,MCO1191948371,Televisor Kalley 32 Atv32hdw Hd Led Plano Sma...,new,MCO20774004,gold_special,https://www.mercadolibre.com.co/televisor-kall...,buy_it_now,MCO,MCO14903,MCO-TELEVISIONS,...,NaN,238038329,TIENDA-OFICIAL-EN-LINEA,36.0,18302.78,0.0,COP,Kalley,NaN,NaN
4,MCO1301778229,Parlante Harman Kardon Onyx Studio 8 Hkos8blkb...,new,MCO22655660,gold_special,https://www.mercadolibre.com.co/parlante-harma...,buy_it_now,MCO,MCO3691,MCO-SPEAKERS,...,NaN,134165426,PDA YSMARTPHONE,36.0,20074.78,0.0,COP,Harman Kardon,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32821,MCO1339878695,Cortadora De Litio Profoil - Andis,new,MCO20714998,gold_special,https://www.mercadolibre.com.co/cortadora-de-l...,buy_it_now,MCO,MCO5411,MCO-HAIR_CLIPPERS_ELECTRIC_SHAVERS_AND_HAIR_TR...,...,NaN,147178293,CERO2121540,36.0,6638.89,0.0,COP,Andis,NaN,NaN
32822,MCO1784215016,Nevera No Frost 370 Litros Brutos - Cr 370,new,MCO28714347,gold_pro,https://articulo.mercadolibre.com.co/MCO-17842...,buy_it_now,MCO,MCO115334,MCO-REFRIGERATORS,...,NaN,388583366,CHALLENGER S.A.S.,12.0,163325.00,0.0,COP,Challenger,NaN,NaN
32823,MCO1253292455,Nevera Bottom Freezer 542 Lts Brutos Black Mab...,new,MCO28517271,gold_pro,https://articulo.mercadolibre.com.co/MCO-12532...,buy_it_now,MCO,MCO115334,MCO-REFRIGERATORS,...,NaN,175444046,MABECOLOMBIASAS,12.0,358325.00,0.0,COP,Mabe,NaN,NaN
32824,MCO1363595409,Nevera No Frost 247 L Brutos Grafito Mabe - Rm...,new,MCO28575716,gold_special,https://articulo.mercadolibre.com.co/MCO-13635...,buy_it_now,MCO,MCO115334,MCO-REFRIGERATORS,...,NaN,175444046,MABECOLOMBIASAS,36.0,46108.33,0.0,COP,Mabe,NaN,NaN
